<a href="https://colab.research.google.com/github/Luensmann/Bachelorarbeit/blob/main/Pre%20Processing/raw2ibo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import json
import re
import spacy
from spacy.tokenizer import Tokenizer

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp.tokenizer = Tokenizer(nlp.vocab, token_match=re.compile(r'\S+').match)

In [ ]:
def open_doc(filename):
    with open(filename) as f:
        corpus = json.load(f)
    f.close
    return corpus

In [ ]:
def write_doc(data, filename):
    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

In [ ]:
def get_entTypes(docEntity):
    entTypes = []
    for ent in docEntity:
        entTypes.append(ent["type"])
    return entTypes

In [ ]:
def get_entLim(docEnts):
    entLim = []
    
    for ent in docEnts:
        entLim.append(ent["begin"])
        entLim.append(ent["end"])
    return entLim

In [ ]:
def get_entTxt(docEnts):
    entTxt = []
    
    for ent in docEnts:
        entTxt.append(ent["text"])
    return entTxt

In [ ]:
def split_text(docText, entLim):
    parts = list(docText[i:j] for i,j in zip(entLim, entLim[1:]+[None]))
    if(len(entLim) > 0 and entLim[0] != 0):
        parts.insert(0, docText[0:entLim[0]])
    return parts

In [ ]:
def sync_nerTags(nerTags, splts, entTxt, entTypes):
    entCnt = 0
    for splt in range(len(splts)):
        if(splts[splt] == entTxt[entCnt]):
            nerTags[splt] = entTypes[entCnt]
            if(entCnt < len(entTypes)-1):
                entCnt += 1
        else: 
            nerTags[splt] = "O"

In [ ]:
def txt2json(splts, docTxt, nerTags, docID):
    id = docID
    tkns = []
    ner_tags2 = []  

    if(len(splts) == 0):
        doc = nlp.tokenizer(docTxt)
        for tkn in doc:
            tkns.append(str(tkn))
            ner_tags2.append(0)
        output = {"id": id, "raw_text": docTxt, "token": tkns, "ner_tags": ner_tags2}
        return output
    
    for splt in range(len(splts)):
        doc = nlp.tokenizer(splts[splt])
        
        for tkn in range(len(doc)):
            tkns.append(str(doc[tkn]))
            if(nerTags[splt] != "O"):
                if(tkn == 0):
                    ner_tags2.append(label2id["B-" + nerTags[splt]])
                else:
                    ner_tags2.append(label2id["I-" + nerTags[splt]])
            else:
                ner_tags2.append(0)     
    
    output = {"id": id, "raw_text": docTxt, "token": tkns, "ner_tags": ner_tags2}
    return output

In [ ]:
'''
labels_Variome = ['O', 'B-disease', 'I-disease', 'B-body-part', 'I-body-part', 'B-mutation', 'I-mutation', 'B-Physiology', 'I-Physiology', 'B-cohort-patient', 'I-cohort-patient', 'B-size', 'I-size', 'B-gender', 'I-gender', 'B-age', 'I-age', 'B-Concepts_Ideas', 'I-Concepts_Ideas', 'B-Disorder', 'I-Disorder', 'B-gene', 'I-gene', 'B-Phenomena', 'I-Phenomena', 'B-ethnicity', 'I-ethnicity']
labels_Amia = ['O', 'B-Gene_protein', 'I-Gene_protein', 'B-DNA_Mutation', 'I-DNA_Mutation', 'B-DNA_modification', 'I-DNA_modification', 'B-locus', 'I-locus', 'B-Mutation', 'I-Mutation', 'B-dbSNP', 'I-dbSNP', 'B-Protein_Mutation', 'I-Protein_Mutation', 'B-RNA', 'I-RNA', 'B-RNA_Mutation', 'I-RNA_Mutation']
labels_SETH = ['O', 'B-Gene', 'I-Gene', 'B-SNP', 'I-SNP', 'B-RS', 'I-RS']
labels_tmvar = ['O', 'B-DNAMutation', 'I-DNAMutation', 'B-ProteinMutation', 'I-ProteinMutation', 'B-SNP', 'I-SNP']


#labels = ['disease', 'body-part', 'mutation', 'Physiology', 'cohort-patient', 'size', 'gender', 'age', 'Concepts_Ideas', 'Disorder', 'gene', 'Phenomena', 'ethnicity'] # Variome
#labels = ['Gene_protein', 'DNA_Mutation', 'DNA_modification', 'locus', 'Mutation', 'dbSNP', 'Protein_Mutation', 'RNA', 'RNA_Mutation'] # Amia
#labels = ['Gene', 'SNP', 'RS'] # SETH
#labels = ['DNAMutation', 'ProteinMutation', 'SNP'] # tmvar
labels = [['DNAMutation', 'ProteinMutation', 'SNP'], ['Gene', 'SNP', 'RS'], ['Gene_protein', 'DNA_Mutation', 'DNA_modification', 'locus', 'Mutation', 'dbSNP', 'Protein_Mutation', 'RNA', 'RNA_Mutation'], ['disease', 'body-part', 'mutation', 'Physiology', 'cohort-patient', 'size', 'gender', 'age', 'Concepts_Ideas', 'Disorder', 'gene', 'Phenomena', 'ethnicity']]

label_names = ["O"]
for label in labels:
    label_names.append("B-" + label)
    label_names.append("I-" + label)

print(label_names)
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}
'''

In [ ]:
def get_labels(docNum):
    labels = [['DNAMutation', 'ProteinMutation', 'SNP'], ['Gene', 'SNP', 'RS'], ['Gene_protein', 'DNA_Mutation', 'DNA_modification', 'locus', 'Mutation', 'dbSNP', 'Protein_Mutation', 'RNA', 'RNA_Mutation'], ['disease', 'body-part', 'mutation', 'Physiology', 'cohort-patient', 'size', 'gender', 'age', 'Concepts_Ideas', 'Disorder', 'gene', 'Phenomena', 'ethnicity']]
    label_names = ["O"]
    for label in labels[docNum]:
        label_names.append("B-" + label)
        label_names.append("I-" + label)
    print(label_names)
    return(label_names)

In [ ]:
import pprint

label_names = get_labels(0)
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}
pprint.pprint(label2id)
pprint.pprint(id2label)
for label in label2id:
    print(label)

In [ ]:
tmvar = ['tmvar-train', 'tmvar-test'] # 0
SETH = ['SETH-train', 'SETH-test'] # 1
amia = ['amia-train', 'amia-test'] # 2
Variome = ['Variome-train', 'Variome-test'] # 3
corpora = [ ['tmvar-train', 'tmvar-test'], ['SETH-train', 'SETH-test'], ['amia-train', 'amia-test'], ['Variome-train', 'Variome-test']]

for docNum in range(4):
    label_names = get_labels(docNum)
    id2label = {i: label for i, label in enumerate(label_names)}
    label2id = {v: k for k, v in id2label.items()}

    for doc in corpora[docNum]:
        print('corpus name:' + doc)
        corpus = open_doc(doc + '.json')
        jsonOut = []
        filename = doc
        fOut = doc + '-ibo.json'
        all_Ents = []
        for document in corpus["documents"]:
            docID = document["document"]["ID"]
            docText = document["document"]["text"]
            docEnts = document["document"]["entities"]

            entLim = get_entLim(docEnts)
            entTypes = get_entTypes(docEnts)

            for ent in entTypes:
                if ent not in all_Ents:
                    all_Ents.append(ent)
            entTxt = get_entTxt(docEnts)
            splts = split_text(docText, entLim)
            nerTags = [0] * len(splts)
            sync_nerTags(nerTags, splts, entTxt, entTypes)
            out = txt2json(splts, docText, nerTags, docID)
            jsonOut.append(out)
        print(len(jsonOut))
        for i in range(len(jsonOut)):

            print("#" + jsonOut[i]["id"])
            for tag in range(len(jsonOut[i]["ner_tags"])):
                if(jsonOut[i]["ner_tags"][tag] != 0):

                    print(jsonOut[i]["token"][tag], jsonOut[i]["ner_tags"][tag])
            print("\n")
    
        print('found ents:', all_Ents)
        write_doc(jsonOut, fOut)